In [39]:
import pandas as pd

In [60]:
dbi_complaints = pd.read_csv('RawData/DBI_Complaints.csv', low_memory=False)

In [5]:
dbi_violations = pd.read_csv('DBI_Violations.csv', low_memory=False)

In [6]:
#group number of violations per complaint 
dbi_violations['num_items']=1
dbi_violation_num =dbi_violations[['num_items','Complaint Number']].groupby(['Complaint Number']).agg('count')
dbi_violation_num.reset_index(inplace=True)
dbi_violation_num.head()
#dbi_violation_num.shape

,Complaint Number,num_items
0,199600074,6
1,199600078,4
2,199600079,9
3,199600081,6
4,199600087,9


In [7]:
dbi_merge = dbi_complaints.merge(dbi_violation_num, left_on='Complaint Number', right_on ='Complaint Number', how ='left')

In [8]:
dbi_merge.shape

(231343, 26)

In [9]:
dbi_merge.head()

,Complaint Number,Date Filed,Date 1st NOV Issued,Date 2nd NOV Issued,Date Final Warning Letter Issued,Date Referred to City Attorney,Director Hearing Date,Last Inspection Date,Date Abated,Closed Date,...,Complaint Description,Status,NOV Type,Receiving Division,Assigned Division,Neighborhoods - Analysis Boundaries,Supervisor District,Zipcode,Location,num_items
0,201623661,06/23/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/30/2017,...,contractor working outside of building permit ...,not active,NaN,Inspection Services,Building Inspection Division,North Beach,3.0,94133.0,"(37.80174384291092, -122.40768328354719)",NaN
1,201576215,10/27/2015,NaN,NaN,NaN,NaN,NaN,11/02/2015,11/06/2015,NaN,...,sink is clogged up.,not active,NaN,Housing Inspection Services,Housing Inspection Services,Mission,9.0,94110.0,"(37.76384499486115, -122.41987268743213)",NaN
2,200920802,09/01/2009,09/04/2009,NaN,NaN,NaN,NaN,09/24/2009,09/28/2009,NaN,...,sewage leak throughout entire building 8/28 on...,not active,complaint,Housing Inspection Services,Housing Inspection Services,Potrero Hill,10.0,94107.0,"(37.76018640761093, -122.38994688869126)",4.0
3,200860886,06/12/2008,NaN,NaN,NaN,NaN,NaN,06/11/2008,06/11/2008,NaN,...,routine inspection of common areas of subject ...,not active,NaN,Housing Inspection Services,Housing Inspection Services,Bernal Heights,9.0,94110.0,"(37.739743698837614, -122.42329615670654)",NaN
4,200671126,07/03/2006,NaN,NaN,NaN,NaN,NaN,NaN,07/05/2006,NaN,...,work w/out a permit at rear deck.,not active,NaN,Building Inspection Division,Building Inspection Division,Bayview Hunters Point,10.0,94124.0,"(37.73616815236982, -122.38652614498953)",NaN


In [37]:
last=pd.to_datetime(dbi_merge['Date Filed'], format='%m/%d/%Y', errors='coerce')
first=pd.to_datetime(dbi_merge['Date 1st NOV Issued'], format='%m/%d/%Y', errors='coerce')

In [38]:
ls=[]
for l, f in zip(last,first):
    ls.append(pd.Timedelta(f-l))

In [12]:
dbi_merge['days_to_resolve']=ls

In [13]:
dbi_merge['complaint_num']=1
dbi_merge['parcel'] = dbi_merge['Block']+dbi_merge['Lot']

In [14]:
#code the number of complaint types per 
dbi_nov_type =dbi_merge[['NOV Type','parcel','complaint_num']].groupby(['NOV Type','parcel']).agg('count')
dbi_nov_type.reset_index(inplace=True)
dbi_nov_type.head()

dbi_nov_type_long=dbi_nov_type.pivot_table(index=['parcel'],columns='NOV Type', values='complaint_num')
dbi_nov_type_long.fillna(0, inplace=True)
dbi_nov_type_long.reset_index(inplace=True)
dbi_nov_type_long.head()

NOV Type,parcel,annual routine,ca task force,complaint,hotel room-room insp,other,routine
0,0011007,0.0,0.0,0.0,0.0,0.0,2.0
1,0011008,0.0,0.0,0.0,0.0,0.0,1.0
2,0013009,0.0,0.0,0.0,0.0,0.0,2.0
3,0014001,0.0,0.0,0.0,0.0,0.0,3.0
4,0020001,0.0,0.0,0.0,0.0,0.0,2.0


In [16]:
#code the number of complaint types per 
dbi_ass_div =dbi_merge[['Assigned Division','parcel','complaint_num']].groupby(['Assigned Division','parcel']).agg('count')
dbi_ass_div.reset_index(inplace=True)
dbi_ass_div.head()


dbi_ass_div_long=dbi_ass_div.pivot_table(index=['parcel'],columns='Assigned Division', values='complaint_num')
dbi_ass_div_long.fillna(0, inplace=True)
dbi_ass_div_long.reset_index(inplace=True)
dbi_ass_div_long.head()

Assigned Division,parcel,Building Inspection Division,Central Permit Bureau,Code Enforcement Section,Department Of Bldg Inspection,Department Of City Planning,Disabled Access Division,Electrical Inspection Division,Fire Prevention Bureau,Health Department,Help Desk / Technical Services,Housing Inspection Services,Major Plan Check Division,Other/outside Agency,Plumbing Inspection Division,Seismic Safety Section
0,0004001,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0005001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0006001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0010001,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,0010002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [20]:
dbi_main=dbi_ass_div_long.merge(dbi_nov_type_long, left_on='parcel', right_on='parcel', how='outer')
dbi_main.shape

(62952, 22)

In [51]:
dbi_violation_items =dbi_merge[['num_items','parcel']].groupby(['parcel']).agg('sum')
dbi_violation_items.reset_index(inplace=True)
dbi_violation_items.fillna(0, inplace=True)
dbi_violation_items.head()

,parcel,num_items
0,0004001,0.0
1,0005001,0.0
2,0006001,0.0
3,0010001,0.0
4,0010002,0.0


In [52]:
dbi_main=dbi_main.merge(dbi_violation_items, left_on='parcel', right_on='parcel', how='outer')

In [54]:
dbi_main.head()

,parcel,Building Inspection Division,Central Permit Bureau,Code Enforcement Section,Department Of Bldg Inspection,Department Of City Planning,Disabled Access Division,Electrical Inspection Division,Fire Prevention Bureau,Health Department,...,Other/outside Agency,Plumbing Inspection Division,Seismic Safety Section,annual routine,ca task force,complaint,hotel room-room insp,other,routine,num_items
0,0004001,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0005001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0006001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0010001,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0010002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [55]:
R2Parcel = pd.read_csv('residential_units.csv')

In [56]:
dbi_violations_clean = R2Parcel.merge(dbi_main,left_on=['Block and Lot Number'], 
                                                      right_on=['parcel'], how='inner')

In [57]:
dbi_violations_clean.shape

(13837, 27)

In [58]:
dbi_violations_clean.head()

,Unnamed: 0,Block and Lot Number,Property Location,Location,parcel,Building Inspection Division,Central Permit Bureau,Code Enforcement Section,Department Of Bldg Inspection,Department Of City Planning,...,Other/outside Agency,Plumbing Inspection Division,Seismic Safety Section,annual routine,ca task force,complaint,hotel room-room insp,other,routine,num_items
0,0,1551051,0000 1725 ANZA ST0000,"(37.7787266984196, -122.466805968567)",1551051,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,37.0
1,2,1053011,2562 2558 SUTTER ST0000,"(37.7851943530233, -122.442546803931)",1053011,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0
2,3,0087013,0000 1654 GRANT AV0000,"(37.8023458331673, -122.407723330705)",0087013,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,5,4078017,0707 0705 HAMPSHIRE ST0000,"(37.7602442525718, -122.407717167359)",4078017,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0
4,14,1166007,2622 2620 MCALLISTER ST0000,"(37.7758429805692, -122.455397897246)",1166007,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0


In [59]:
dbi_violations_clean.to_csv('dbi_violations_clean.csv')